In [1]:
from nltk import FreqDist
import pandas as pd

def obtenerPalabrasModelo(nombre_archivo, tag, positive, min_largo, min_intensidad, clase):
  tweet_df = pd.read_csv(nombre_archivo, index_col=0)
  lista_tokens = []
  lista_tokens.clear()
  lista_palabras = []
  lista_palabras.clear()
  for tweet, etiqueta, cl in zip(tweet_df['clean_tweet'], tweet_df[tag], tweet_df[clase]):
    if cl == 'Positivo' or cl == 'Negativo':
      lista_palabras = []
      etiqueta = etiqueta.split(",")
      tags = []
      tags.clear()
      for valor in etiqueta:
        valor = valor.replace(" ", "");
        valor = valor.replace("[", "");
        valor = valor.replace("]", "");
        tags.append(valor)
      tokens = int(tags[2])
      if tokens >= min_largo:
        if (positive and tags[0]>tags[1]) or ((not positive) and tags[0]<tags[1]):
          porcentaje = 0
          if positive:
            porcentaje = (int(tags[0])/tokens)*100
          else:
            porcentaje = (int(tags[1])/tokens)*100
          if porcentaje >= min_intensidad:
            palabras = tweet.split(",")
            for palabra in palabras:
              palabra = palabra.replace("'", "");
              palabra = palabra.replace(" ", "");
              palabra = palabra.replace("[", "");
              palabra = palabra.replace("]", "");
              lista_palabras.append(palabra)
            if len(lista_palabras) > 0:
              lista_tokens.append(lista_palabras)
  return lista_tokens

tags = ['clean_tag','lemma_tag']
clase = ['clean_class','lemma_class']
archivo_base = 'Tweets-2'
pos = 0
min_largo = 0
min_intensidad = 0
palabras_positivas_recopiladas = []
palabras_positivas_recopiladas.clear()
palabras_negativas_recopiladas = []
palabras_negativas_recopiladas.clear()
archivo_base = archivo_base + '.csv'
palabras_positivas_recopiladas = obtenerPalabrasModelo(archivo_base, tags[pos], True, min_largo, min_intensidad, clase[pos])
palabras_negativas_recopiladas = obtenerPalabrasModelo(archivo_base, tags[pos], False, min_largo, min_intensidad, clase[pos])

In [2]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(palabras_positivas_recopiladas)
all_neg_words = get_all_words(palabras_negativas_recopiladas)
print(all_pos_words)
        
freq_dist_pos = FreqDist(all_pos_words)
#print("Palabras Positivas mas Frecuentes")
list_popular_pos = freq_dist_pos.most_common(50)
#print(list_popular_pos)
#print("")
freq_dist_neg = FreqDist(all_neg_words)
#print("Palabras Negativas mas Frecuentes")
list_popular_neg = freq_dist_neg.most_common(50)
#print(list_popular_neg)
#print("")

<generator object get_all_words at 0x0000015B328BEC10>


In [24]:
lista_pos_bigramas = []
for palabras in palabras_positivas_recopiladas[:5000]:
    bigramas = []
    bigramas.clear()
    for palabra in palabras:
        if len(bigramas) == 0:
            bigramas.append(' ' + palabra)
            anterior = palabra
            continue
        else:
            bigramas.append(anterior + ' ' + palabra)
            anterior = palabra
    bigramas.append(palabra + ' ')
    lista_pos_bigramas.append(bigramas)

lista_neg_bigramas = []
for palabras in palabras_negativas_recopiladas[:5000]:
    bigramas = []
    bigramas.clear()
    for palabra in palabras:
        if len(bigramas) == 0:
            bigramas.append(' ' + palabra)
            anterior = palabra
            continue
        else:
            bigramas.append(anterior + ' ' + palabra)
            anterior = palabra
    bigramas.append(palabra + ' ')
    lista_neg_bigramas.append(bigramas)

all_bi_pos_words = get_all_words(lista_pos_bigramas)
all_bi_neg_words = get_all_words(lista_neg_bigramas)
    
freq_dist_bi_pos = FreqDist(all_bi_pos_words)
#print("Par de Palabras Positivas mas Frecuentes")
list_popular_bi_pos = freq_dist_bi_pos.most_common(50)
#print(lista_popular_bi_pos)
#print("")
freq_dist_bi_neg = FreqDist(all_bi_neg_words)
#print("Par de Palabras Negativas mas Frecuentes")
list_popular_bi_neg = freq_dist_bi_neg.most_common(50)
#print(lista_popular_bi_neg)

In [25]:
tuplas = []
tuplas.clear()
for i in range(50):
    tuplas.append([str(i+1),list_popular_pos[i],list_popular_neg[i],list_popular_bi_pos[i],list_popular_bi_neg[i]])
df = pd.DataFrame(tuplas, columns = ['indice','unigrama_pos', 'unigrama_neg', 'bigrama_pos', 'bigrama_neg'])  
df.to_csv('Hashtags.csv', index=True, header=True)

In [2]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

positive_tokens_for_model = get_tweets_for_model(palabras_positivas_recopiladas)
negative_tokens_for_model = get_tweets_for_model(palabras_negativas_recopiladas)

In [3]:
import random
random.seed(999)

positive_tag = [(tweet_dict, "Positive")
                     for tweet_dict in palabras_positivas_recopiladas]
negative_tag = [(tweet_dict, "Negative")
                     for tweet_dict in palabras_negativas_recopiladas]

tags = positive_tag + negative_tag
random.shuffle(tags)
corte = int(len(tags)*0.7)
train = tags[corte:]
test = tags[:corte]

In [4]:
x_train =[]
x_train.clear()
y_train =[]
y_train.clear()
for data in train:
    text = ''
    for d in data[0]:
        if text == '':
            text += d
        else:
            text += ' ' + d    
    x_train.append(text)
    y_train.append(data[1])    

x_test =[]
x_test.clear()
y_test =[]
y_test.clear()
for data in test:
    text = ''
    for d in data[0]:
        if text == '':
            text += d
        else:
            text += ' ' + d  
    x_test.append(text)
    y_test.append(data[1])

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(x_train)
vectorizer.vocabulary_
# vectorizer.transform(x_train).toarray()
estadisticos = ["Precision", "Recall", "F-Measure", "Clasificaciones"]

In [ ]:
NBC = MultinomialNB()
NBC.fit(vectorizer.transform(x_train).toarray(),y_train)
y_pred = NBC.predict(vectorizer.transform(x_test).toarray())
print("Estadisticos")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
results = metrics.precision_recall_fscore_support(y_test, y_pred)
i = 0
for res in results:
    print(estadisticos[i] + ": " + str(res))
    i = i+1

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf.fit(vectorizer.transform(x_train).toarray(),y_train)
y_pred = clf.predict(vectorizer.transform(x_test).toarray())
print("Estadisticos")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
results = metrics.precision_recall_fscore_support(y_test, y_pred)
i = 0
for res in results:
    print(estadisticos[i] + ": " + str(res))
    i = i+1

In [ ]:
LogisticRegression_classifier = LogisticRegression()
LogisticRegression_classifier.fit(vectorizer.transform(x_train).toarray(),y_train)
y_pred = LogisticRegression_classifier.predict(vectorizer.transform(x_test).toarray())
print("Estadisticos")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
results = metrics.precision_recall_fscore_support(y_test, y_pred)
i = 0
for res in results:
    print(estadisticos[i] + ": " + str(res))
    i = i+1

In [ ]:
SVC_classifier = SVC(max_iter=100)
SVC_classifier.fit(vectorizer.transform(x_train).toarray(),y_train)
y_pred = SVC_classifier.predict(vectorizer.transform(x_test).toarray())
print("Estadisticos")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
results = metrics.precision_recall_fscore_support(y_test, y_pred)
i = 0
for res in results:
    print(estadisticos[i] + ": " + str(res))
    i = i+1

In [7]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(2, 2), random_state=1, max_iter=100)
clf.fit(vectorizer.transform(x_train).toarray(), y_train)
y_pred = clf.predict(vectorizer.transform(x_test).toarray())
print("Estadisticos")
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
results = metrics.precision_recall_fscore_support(y_test, y_pred)
i = 0
for res in results:
    print(estadisticos[i] + ": " + str(res))
    i = i+1

C:\Users\pablo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Estadisticos
Accuracy: 0.6678196780611322
Precision: [0.67335711 0.66340135]
Recall: [0.61481929 0.71794308]
F-Measure: [0.64275815 0.68959544]
Clasificaciones: [21499 22733]
